In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import pickle

DATA_FILE = 'data/data.pkl'
with open(DATA_FILE, 'rb') as f:
    Cs = pickle.load(f)
    
DATA_FILE2 = 'data/labels.pkl'  
with open(DATA_FILE2, 'rb') as f:
    Ys = pickle.load(f)

In [46]:
import collections

collections.Counter(Ys[0])

Counter({1.0: 741, 5.0: 2115, 4.0: 1028, 3.0: 519, 2.0: 396})

In [47]:
print(f"Train Size: {len(Cs[0])}")
print(f"Dev Size:   {len(Cs[1])}")
print(f"Test Size:  {len(Cs[2])}")

Train Size: 4799
Dev Size:   599
Test Size:  600


In [48]:
candidate = Cs[0][0]
print(f"Sentence:\n{candidate.text}")
print(f"Candidate:\nX: {candidate[0]}\nY: {candidate[1]}")

Sentence:
Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.
Candidate:
X: EntityMention(doc_id=1: 'To'(0:2)
Y: EntityMention(doc_id=1: 'l'(4:5)


In [49]:
from babble.utils import ExplanationIO

FILE = "my_explanations.tsv"
exp_io = ExplanationIO()
explanations = exp_io.read(FILE)

Read 967 explanations from my_explanations.tsv


In [ ]:
# del explanations[0]

In [7]:
print(explanations)

[Explanation(LF_1: 5, "the word "corporate" or "blowout" or "girl" or "Cardenas" or "outfit" or are in the sentence"), Explanation(LF_2: 5, "the word "nice" or "assistants" or "knew" or "feel" or "helped" or are in the sentence"), Explanation(LF_4: 1, "the word "years" or "squealing" or "properly" or "refund" or "gentleman" or are in the sentence"), Explanation(LF_6: 3, "the word "simple" or "finished" or "fresh" or "tables" or "tea" or are in the sentence"), Explanation(LF_17: 3, "the word "cheese" or "cocktails" or "jade" or "eating" or "recommend" or are in the sentence"), Explanation(LF_41: 3, "the word "price" or "places" or "yogurt" or "compare" or "Piccadilly" or are in the sentence"), Explanation(LF_43: 5, "the word "mi" or "checking" or "toasted" or "banh" or "chips" or are in the sentence"), Explanation(LF_55: 4, "the word " " or "nice" or "dan" or "good" or "wide" or are in the sentence"), Explanation(LF_65: 1, "the word "wo" or "SEWING" or "SHOP" or "TAKEN" or "repaired" or

In [50]:
from babble import Babbler
babbler = Babbler(Cs, Ys)

Grammar construction complete.


In [51]:
babbler.apply(explanations, split=0)     

Building list of target candidate ids...
Collected 1 unique target candidate ids from 967 explanations.
Gathering desired candidates...
Could not find 1 target candidates with the following mention_ids (first 5):
-2939145380604818476
Found 0/1 desired candidates
Linking explanations to candidates...
Linked 0/967 explanations
963 explanation(s) out of 967 were parseable.
1387 parse(s) generated from 967 explanation(s).
965 parse(s) remain (422 parse(s) removed by DuplicateSemanticsFilter).
Note: 965 LFs did not have candidates and therefore could not be filtered.
965 parse(s) remain (0 parse(s) removed by ConsistencyFilter).
Applying labeling functions to investigate labeling signature.
[========================================] 100%

965 parse(s) remain (0 parse(s) removed by UniformSignatureFilter: (0 None, 0 All)).
965 parse(s) remain (0 parse(s) removed by DuplicateSignatureFilter).
963 parse(s) remain (2 parse(s) removed by LowestCoverageFilter).
Added 963 parse(s) from 963 explana

In [52]:
Ls = []
for split in [0,1,2]:
    L = babbler.get_label_matrix(split)
    Ls.append(L)

Retrieved label matrix for split 0: L.nnz = 1164233, L.shape = (4799, 963)
Retrieved label matrix for split 1: L.nnz = 140196, L.shape = (599, 963)
Retrieved label matrix for split 2: L.nnz = 142285, L.shape = (600, 963)


In [53]:
Ls[0]

<4799x963 sparse matrix of type '<class 'numpy.int64'>'
	with 1164233 stored elements in Compressed Sparse Row format>

In [54]:
from metal import LabelModel

label_aggregator = LabelModel(5)
label_aggregator.train(Ls[0], n_epochs=250, lr=0.001)
label_aggregator.score(Ls[1], Ys[1])

Computing O...
Estimating \mu...


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


[E:0]	Train Loss: 29740.828


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

[E:10]	Train Loss: 1038.262


...

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

[E:240]	Train Loss: 100.305


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

[E:249]	Train Loss: 99.517
Finished Training
Accuracy: 0.464


0.46410684474123537

In [55]:
Y_p = label_aggregator.predict(Ls[0])

In [56]:
print(Y_p)

[5 5 5 ... 5 5 5]


In [57]:
for y in Y_p:
    if y == 1:
        print(y)

In [58]:
wrong_indexes = []
i = 0
for right,wrong in zip(Ys[0],Y_p):
    if int(right) == wrong:
        wrong_indexes.append(i)
    i = i + 1

In [59]:
print(len(wrong_indexes))

2115


In [60]:
with open("Ls.pkl", 'wb') as f:
    pickle.dump(Ls, f)
    
with open("Y_p.pkl", 'wb') as f:
    pickle.dump(Y_p, f)
    
with open("data/wrong_indexes.pkl", 'wb') as f:
    pickle.dump(wrong_indexes, f)